In [9]:
import torch
# 设置随机数种子
torch.manual_seed(666)
print(torch.initial_seed()) # 查看随机数种子 666

g_2 = torch.manual_seed(0)
print(g_2.initial_seed()) # 0

# 获取默认的 Generator 实例
g_1 = torch.default_generator
print(g_1.initial_seed()) # 0

# 结果为 True
print(g_1 is g_2) # True

666
0
0
True


In [19]:
# 手动创建的随机数生成器
g3 = torch.Generator()
print(g3.initial_seed()) # 67280421310721
g3.manual_seed(1)
print(g3.initial_seed()) # 1

67280421310721
1


In [32]:
# 使用默认的随机数生成器
torch.manual_seed(1)

# 结果 tensor([0, 4, 2, 3, 1])
print(torch.randperm(5))
print(torch.default_generator.device) # cpu

# 手动创建随机数生成器
G = torch.Generator(device='cuda')
G.manual_seed(1)
print(G.initial_seed()) # 1

# 结果 tensor([0, 4, 2, 3, 1])
print(torch.randperm(5, generator=G, device='cuda'))
print(G.device) # cuda

tensor([0, 4, 2, 3, 1])
cpu
1
tensor([1, 3, 2, 4, 0], device='cuda:0')
cuda


In [34]:
# 从std::random_device或当前时间获取不确定的随机数并将其用作生成器的种子
G.seed()
print(G.initial_seed()) 

3710802736670458


In [41]:
kk = torch.Generator()
kk.seed()
print(kk.initial_seed()) # 576724579900700

a=kk.get_state() # 查看该类的状态码
# print(G.get_state())
print('the id of kk is: ',id(kk)) # the id of kk is:  2128861923824

g_cpu_other = torch.Generator()
print(g_cpu_other.initial_seed()) # 67280421310721
g_cpu_other.set_state(kk.get_state())

print(g_cpu_other.initial_seed()) # 576724579900700

print('the id of g_cpu_other is: ',id(g_cpu_other)) # the id of g_cpu_other is:  2128861593168

576724579900700
the id of kk is:  2128861923824
67280421310721
576724579900700
the id of g_cpu_other is:  2128861593168


## part2 实用随机采样器

#### 2.1 伯努利采样器

In [48]:
# 伯努利采样器
a = torch.empty(3, 3).uniform_(0, 1)  # generate a uniform random matrix with range [0, 1]
print(a)
print(torch.bernoulli(a))

a = torch.ones(3, 3) # probability of drawing "1" is 1
print(torch.bernoulli(a))

a = torch.zeros(3, 3) # probability of drawing "1" is 0
print(torch.bernoulli(a))

tensor([[0.4452, 0.0193, 0.2616],
        [0.7713, 0.3785, 0.9980],
        [0.9008, 0.4766, 0.1663]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [1., 1., 0.]])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


In [68]:
a = torch.empty(3, 3).uniform_(0, 1)  # generate a uniform random matrix with range [0, 1]
print(a)
print(torch.bernoulli(a, generator=kk))

tensor([[0.1478, 0.7444, 0.1408],
        [0.3854, 0.8637, 0.8960],
        [0.9729, 0.3985, 0.1114]])
tensor([[0., 0., 0.],
        [1., 1., 1.],
        [1., 1., 1.]])


In [100]:
a1=torch.ones(3, 3)
b1=torch.empty(3, 3).uniform_(0, 1)
a1.bernoulli_(b1)
print(a1)

c1=torch.randint_like(a1,10,device='cuda')
c1.bernoulli_(0.99)
print(c1)

tensor([[1., 0., 1.],
        [0., 0., 0.],
        [0., 0., 1.]])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


#### 2.2正态分布采样器

In [117]:
a2=torch.normal(mean=torch.arange(1., 11.), std=torch.arange(1, 0, -0.1))
print(a2)

b2=torch.normal(mean=250, std=torch.arange(1., 6.,device='cuda'))
print(b2)

c2=torch.normal(mean=torch.arange(1., 6.,device='cuda'), std=0.1)
print(c2)

d2=torch.normal(mean=torch.arange(1., 13.).reshape(3,4), std=torch.arange(1., 13.).reshape(1,12))
print(d2)

e2 = torch.normal(0, 88, size=(1, 4))
print(e2)

tensor([ 3.0170,  2.7382,  2.4615,  3.6176,  4.7924,  5.6880,  6.6683,  7.7693,
         9.1002, 10.0412])
tensor([251.3551, 251.3911, 252.0018, 245.8633, 248.5067], device='cuda:0')
tensor([0.8146, 2.0670, 2.9554, 3.9749, 5.0211], device='cuda:0')
tensor([[ 1.1248,  3.7747, -1.0588,  6.6303],
        [ 1.1274,  5.7302, 17.1167,  8.3153],
        [ 8.6236,  2.6557, -9.5730,  6.1679]])
tensor([[  21.9903,    8.2003,  -41.6389, -126.4552]])


#### 2.3 泊松分布采样器

In [122]:
rates = torch.rand(4, 4, device='cuda') * 5  # rate parameter between 0 and 5
torch.poisson(rates)

tensor([[1., 1., 1., 2.],
        [6., 6., 2., 1.],
        [1., 4., 0., 3.],
        [1., 2., 2., 1.]], device='cuda:0')

#### 2.4多项式分布采样器

In [178]:
weights = torch.tensor([0, 5, 3, 0], dtype=torch.float) # create a tensor of weights
print(torch.multinomial(weights, 2))  
print(torch.multinomial(weights, 4))

print(torch.multinomial(weights, 4, replacement=True))


tensor([1, 2])
tensor([1, 2, 0, 3])
tensor([2, 1, 2, 1])
